### Diff for bike flows (time evolution)

Given two months, this notebook computes what has changed on the most *relevant* flows from one month to another:
* which new relevant flows have appeared (plotted in green)
* which old relevant flows are not relevant anymore (plotted in red)

Here, **relevant** means *flows belonging to the 4th tier in that month*.

In [1]:
import glob
from ipywidgets import interact_manual, widgets
from IPython.core.display import display, HTML

import bikescience.grid as grid
import bikescience.load_trips as tr
import bikescience.flow_diffs as diffs
import bikescience.stations as st

In [2]:
def plot_diffs(start, start_period, end, end_period, grid_size, perc_below):
    print('Calculating...')
    boston_grid = grid.Grid(n=grid_size)
    start_filter = tr.period_functions[start_period]
    end_filter = tr.period_functions[end_period]
    start_trips = start_filter(tr.load_trips_file(start))
    print('start_trips')
    display(start_trips)
    end_trips = end_filter(tr.load_trips_file(end))
    # display(start_trips)
    fmap = diffs.diffs_map(start_trips, end_trips, boston_grid, stations, stations_distances, perc_below)
    print('Done.')
    file = 'maps/diffs.html'
    fmap.save(file)
    display(HTML('Saved at <a href="' + file + '" target="_blank">' + file + '</a>'))
    display(fmap)

In [3]:
import pandas as pd
trips = pd.read_csv('../data/trips/loaded_trips/trips_few.csv')
trips

,date,Index,tripduration,start_station_name_old,start station id,starttime,end_station_name_old,end station id,stoptime,per_day,hour,week_day,weekend,holiday,trip_id,birth_year,age,distance_in_meters_bicycle,start station name,end station name
0,2018-01-26,954234,114,HQ Test Station,NaN,2018-01-26 15:35:17,HQ Test Station,NaN,2018-01-26 15:37:11,2018-01-26,15,4,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-27,954236,3117,15 - Rua Prof. Artur Ramos,15.0,2018-01-27 12:15:21,10 - Rua Campo Verde,10.0,2018-01-27 13:07:18,2018-01-27,12,5,True,False,NaN,NaN,NaN,1271.0,Rua Prof. Artur Ramos,Rua Campo Verde
2,2018-01-27,954237,550,10 - Rua Campo Verde,10.0,2018-01-27 13:10:05,10 - Rua Campo Verde,10.0,2018-01-27 13:19:15,2018-01-27,13,5,True,False,NaN,NaN,NaN,0.0,Rua Campo Verde,Rua Campo Verde
3,2018-01-27,954238,4480,10 - Rua Campo Verde,10.0,2018-01-27 13:20:35,27 - Rua Min. Jesuíno Cardoso,27.0,2018-01-27 14:35:15,2018-01-27,13,5,True,False,NaN,NaN,NaN,NaN,Rua Campo Verde,NaN
4,2018-01-27,954239,125,27 - Rua Min. Jesuíno Cardoso,27.0,2018-01-27 15:02:36,27 - Rua Min. Jesuíno Cardoso,27.0,2018-01-27 15:04:41,2018-01-27,15,5,True,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2022-04-30,8474839,17209,40 - Brasília,40.0,2022-04-30 17:20:45,40 - Brasília,40.0,2022-04-30 22:07:34,2022-04-30,17,5,True,False,13979475BikeSampa,1994-01-01,28.0,0.0,Brasília,Brasília
296,2022-04-30,8474840,4914,165 - Cemitério Consolação,165.0,2022-04-30 16:38:06,71 - Pamplona,71.0,2022-04-30 18:00:00,2022-04-30,16,5,True,False,13978958BikeSampa,1981-01-01,41.0,2690.0,Cemitério Consolação,Pamplona
297,2022-04-30,8474841,1256,320 - Av. Jandira,320.0,2022-04-30 17:43:02,98 - Sampaio Viana,98.0,2022-04-30 18:03:58,2022-04-30,17,5,True,False,13979828BikeSampa,1975-01-01,47.0,5225.0,Jandira,Sampaio Viana
298,2022-04-30,8474843,14599,344 - ROCHAVERÁ,344.0,2022-04-30 18:08:01,114 - Rua Alvorada,114.0,2022-04-30 22:11:20,2022-04-30,18,5,True,False,13980268BikeSampa,2000-01-01,22.0,4396.0,Rochaverá,Rua Alvorada


In [4]:
!pip install rtree
!pip install pygeos


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# folder = '../data/boston/bike-trips/'
# files = list(map(lambda f: (f.replace(folder, '')[:6], f), glob.glob(folder + '*')))
# files.sort()
# files = list(map(lambda f: (f[0][4:6] + '/' + f[0][:4], f[1]), files))

data_folder = '../data/'

files = list([data_folder + 'trips/loaded_trips/trips_few.csv'])

stations = st.load(data_folder + 'stations/treated_data/2021.11.08_Endereços-BikeSAMPA.csv')
stations_distances = st.distances(data_folder + 'stations/distance/stations_distance.csv')

im = interact_manual(
    plot_diffs, 
    start=widgets.Dropdown(options=files), 
    start_period=widgets.Dropdown(options=[('entire period', 0), ('morning', 1), ('lunchtime', 2), ('afternoon', 3)]),
    end=widgets.Dropdown(options=files),
    end_period=widgets.Dropdown(options=[('entire period', 0), ('morning', 1), ('lunchtime', 2), ('afternoon', 3)]),
    grid_size=widgets.SelectionSlider(options=[10, 20, 30, 40], value=20),
    perc_below=widgets.IntSlider(min=0, max=100, value=20, step=1)
)
im.widget.children[6].description = 'Plot Differences'

interactive(children=(Dropdown(description='start', options=('../data/trips/loaded_trips/trips_few.csv',), val…

In [6]:
files

['../data/trips/loaded_trips/trips_few.csv']